# EMAIL SPAM FILTERING - NAIVE BAYES

In [ ]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['message'], data['class'], test_size=0.33, random_state=42)
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(X_train.values)

classifier = MultinomialNB()
targets = y_train.values
classifier.fit(counts, targets)

In [ ]:
examples = ['Free Money now!!!', "how about a game of cricket tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

In [ ]:
l=[]
for i in X_test:
    l.append(i)
l_counts = vectorizer.transform(l)
predicts = classifier.predict(l_counts)
correct=0
wrong=0
for i,j in zip(predicts,y_test):
    if(i==j):
        correct=correct+1
    else:
        wrong=wrong+1
print(correct,wrong)

In [ ]:
#ERROR RATE:
accuracy = (abs(correct-len(l))/len(l))*100
print("Error Rate:",str(accuracy),"%","\nACCURACY:",str(100-accuracy),"%")

In [ ]:
import imaplib
import email
import pandas as pd

host = 'imap.gmail.com'
username = 'xxxxxxxxxxxxxxxxx@gmail.com'
password = 'enteryourpassword'


def get_inbox():
    mail = imaplib.IMAP4_SSL(host)
    mail.login(username, password)
    mail.select("inbox")
    _, search_data = mail.search(None, 'ALL')
    my_message = []
    for num in search_data[0].split():
        l=[]
        email_data = {}
        _, data = mail.fetch(num, '(RFC822)')
        # print(data[0])
        _, b = data[0]
        email_message = email.message_from_bytes(b)
        for header in ['subject', 'to', 'from', 'date']:
            #print("{}: {}".format(header, email_message[header]))
            email_data[header] = email_message[header]
        for part in email_message.walk():
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True)
                email_data['body'] = body.decode()
        l.append(email_message['subject'])
        l.append(email_message['to'])
        l.append(email_message['from'])
        l.append(email_message['date'])
        l.append(email_data['body'])
        my_message.append(l)
    return my_message


if __name__ == "__main__":
    cols = ['subject','to','from','date','body']
    my_inbox = get_inbox()
    df = pd.DataFrame(my_inbox,columns=cols)
    print(df.head())

In [ ]:
emailpre = df['body']
emailpre_counts = vectorizer.transform(emailpre)
predictemail = classifier.predict(emailpre_counts)
df2 = pd.DataFrame({'from':df['from'],'predicted':predictemail,'body':df['body']})
df2

In [ ]:
print("Number of Ham")
df2[df2["predicted"] == 'ham'].count()

In [ ]:
print("Number of Spam")
df2[df2["predicted"] == 'spam'].count()